In [10]:
import pyterrier as pt
import ir_datasets
import os

In [12]:
# pt.init()

In [15]:
ds = pt.get_dataset("irds:disks45/nocr/trec-robust-2004")
print(ds)

INDEX_DIR = "./robust04_index_customfeatures"
os.makedirs(INDEX_DIR, exist_ok=True)

INDEX_DIR = os.path.abspath("./robust04_index_customfeatures")

IRDSDataset('disks45/nocr/trec-robust-2004')


## new index part

In [16]:
META_FIELD_LENGTHS = {"docno": 20, "body": 10000}
INDEXER_PROPERTIES = {"meta.reverse.keys": "docno,body"}

indexer = pt.IterDictIndexer(
    INDEX_DIR,

    text_attrs = ['body'],
    meta = META_FIELD_LENGTHS,
    properties = INDEXER_PROPERTIES
)

index_ref = indexer.index(ds.get_corpus_iter())

print("Finished")


disks45/nocr/trec-robust-2004 documents:  25%|██▍       | 130506/528155 [03:28<31:18, 211.65it/s]

21:39:56.302 [main] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (FR940104-2-00103) - further warnings are suppressed



disks45/nocr/trec-robust-2004 documents: 100%|█████████▉| 527970/528155 [13:48<00:00, 637.43it/s]


21:50:16.230 [main] WARN org.terrier.structures.indexing.Indexer -- skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "C:\Users\nienk\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyterrier\utils.py", line 139, in _wrapper
    return fn(*args, **kwargs)
  File "C:\Users\nienk\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyterrier\terrier\java.py", line 295, in next
    self._next = next(self._it, StopIteration)
                 ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nienk\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyterrier\terrier\index.py", line 616, in <genexpr>
    return ({f: doc[f] for f in all_cols} for doc in it)
                                                     ^^
  File "C:\Users\nienk\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyterrier\datasets\_irds.py", line 47, in gen
    for doc in it:
               ^^
  File "C:\Users\nienk\AppData\L

21:50:48.113 [main] WARN org.terrier.structures.indexing.Indexer -- Indexed 126 empty documents
Finished


In [17]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics())

21:50:51.833 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 673,3 MiB of memory would be required.
Number of documents: 527970
Number of terms: 518597
Number of postings: 81882805
Number of fields: 0
Number of tokens: 141200305
Field names: []
Positions:   false



In [18]:
meta_index = index.getMetaIndex()

print(meta_index.getKeys())
print(list(meta_index.getReverseKeys()))

['docno', 'body']
['docno']


## XG Boost Part

In [20]:
import os
import pyterrier as pt
import pandas as pd
import xgboost as xgb
import re

In [22]:
#Function 1
def count_term_in_body_unique(row, meta_index):
    """
    Counts how many unique terms in the query appear in the document body.
    """
    docno = row["docno"]
    docid = meta_index.getDocument("docno", docno)

    if docid == -1:
        return 0

    body = meta_index.getItem("body", docid)

    body_terms = set(body.lower().split())
    query_terms = set(row["query"].lower().split())

    count = sum(1 for term in query_terms if term in body_terms)
    return count

In [23]:
#Function 2
def count_term_in_body_full(row, meta_index):
    """
    Counts the total occurrences of query terms in the document body.
    """
    docno = row["docno"]
    docid = meta_index.getDocument("docno", docno)

    if docid == -1:
        return 0

    body = meta_index.getItem("body", docid) or ""

    body_terms = body.lower().split()
    query_terms = row["query"].lower().split()

    # Count total occurrences
    count = sum(body_terms.count(term) for term in query_terms)
    return count

In [25]:
#Testing the functions on an example:
docid = 0
body = meta_index.getItem("body", docid)
docno = meta_index.getItem("docno", docid)

# print("DOCNO:", docno)
# print("BODY:", body[:300])

test_row = {
    "docno": docno,
    "query": "example query relative. the and"
}


count_unique = count_term_in_body_unique(test_row, meta_index)
print("\nNumber of unique query terms in document:", count_unique)

count_full = count_term_in_body_full(test_row, meta_index)
print("\nNumber of query terms in document in total:", count_full)


Number of unique query terms in document: 2

Number of query terms in document in total: 60


In [26]:
custom_feature_transformer_unique = pt.apply.doc_score(
    lambda row: count_term_in_body_unique(row, meta_index)
)

custom_feature_transformer_full = pt.apply.doc_score(
    lambda row: count_term_in_body_full(row, meta_index)
)

In [27]:
topics = ds.get_topics().rename(columns={"title": "query"})
topics["query"] = topics["query"].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', ' ', x))
qrels = ds.get_qrels()

split = int(len(topics) * 0.8)
train_topics = topics.iloc[:split]
valid_topics = topics.iloc[split:]
train_qrels = qrels[qrels["qid"].isin(train_topics["qid"])]
valid_qrels = qrels[qrels["qid"].isin(valid_topics["qid"])]


bm25 = pt.terrier.Retriever(index_ref, wmodel="BM25")
tf = pt.terrier.Retriever(index_ref, wmodel="Tf")
pl2 = pt.terrier.Retriever(index_ref, wmodel="PL2")

pipeline = bm25 >> (tf ** pl2 ** custom_feature_transformer_unique ** custom_feature_transformer_full)

xgb_ranker = xgb.XGBRanker(
    objective="rank:ndcg",
    learning_rate=0.1,
    gamma=1.0,
    min_child_weight=0.1,
    max_depth=6,
    random_state=42
)

There are multiple query fields available: ('title', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
21:54:00.833 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 673,3 MiB of memory would be required.
21:54:00.993 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 673,3 MiB of memory would be required.
21:54:01.132 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 673,3 MiB of memory would be required.


In [28]:
ltr_pipeline = pipeline >> pt.ltr.apply_learned_model(xgb_ranker, form="ltr")

ltr_pipeline.fit(train_topics, train_qrels, valid_topics, valid_qrels)

In [29]:
results = ltr_pipeline.transform(valid_topics)
results.to_csv("ltr_results_nn.csv", index=False)
print(results.head(10))

   qid   docid          docno     score                   query  \
0  651  335499    FT941-13121  1.019730  U S  ethnic population   
1  651   36640    FBIS3-36641  0.811004  U S  ethnic population   
2  651  111209    FBIS4-49817  0.859434  U S  ethnic population   
3  651  222897     FT922-2504  0.884701  U S  ethnic population   
4  651   93471    FBIS4-32079  0.433843  U S  ethnic population   
5  651  251148     FT924-1179  0.410839  U S  ethnic population   
6  651  110379    FBIS4-48987  0.341767  U S  ethnic population   
7  651  101239    FBIS4-39847 -0.066448  U S  ethnic population   
8  651  230420     FT923-9432  0.341767  U S  ethnic population   
9  651  517050  LA120190-0057  0.363532  U S  ethnic population   

                                         description  \
0  How is the ethnic make-up of the U.S. populati...   
1  How is the ethnic make-up of the U.S. populati...   
2  How is the ethnic make-up of the U.S. populati...   
3  How is the ethnic make-up of the U.

In [30]:
evalution = pt.Experiment([ltr_pipeline],valid_topics,valid_qrels, eval_metrics=["map", "ndcg_cut_10"])
print(pd.DataFrame(evalution))

                                                name       map  ndcg_cut_10
0  (TerrierRetr(BM25) >> (TerrierRetr(Tf) ** Terr...  0.264017     0.428892
